In [1]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
from underthesea import word_tokenize

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
stopwords_path = "Data/Data_ref/vietnamese-stopwords.txt"

In [4]:
# df = pd.read_csv('Data/Data_dat_dai/data_after_preprocessing.csv', index_col=0)
# df.head()

In [5]:
# X_train, X_test, Y_train, Y_test= model_selection.train_test_split(df['text'],df['label'],test_size=0.2,random_state=1)

In [6]:
def read_splitted_data():

    df_train = pd.read_csv('Data/Data_dat_dai/df_train_after.csv')
    df_test = pd.read_csv('Data/Data_dat_dai/df_test_after.csv')


    X_train = df_train[['text']]
    Y_train = df_train[['label']]

    X_test = df_test[['text']]
    Y_test = df_test[['label']]   

    return X_train, Y_train, X_test, Y_test

In [7]:
def tf_idf_config(X_train, X_test):

    Tfidf_vect = TfidfVectorizer(max_features=1000)
    Tfidf_vect.fit(X_train['text'].to_list())
    tfidf_train = Tfidf_vect.transform(X_train['text'])
    tfidf_test = Tfidf_vect.transform(X_test['text'])

    tfidf_train = tfidf_train.toarray()
    tfidf_test = tfidf_test.toarray()

    return Tfidf_vect, tfidf_train, tfidf_test

In [8]:
def model_selection(model_name, tfidf_train, tfidf_test, Y_train, Y_test):
    if (model_name == "SVM"):
        model = svm.SVC(C=1.0, kernel='linear', gamma='auto', degree = 3)
        model.fit(tfidf_train,Y_train)
        # predict the labels on validation dataset
        predictions_SVM = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)  
        print(classification_report(Y_test,predictions_SVM)) 
        return model    

    elif (model_name == "LGBM"):

        model = LGBMClassifier()
        model.fit(tfidf_train,Y_train)
        predictions_lgbm = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, Y_test)*100)
        print(classification_report(Y_test,predictions_lgbm))

        return model

    elif (model_name == "Random Forest"):

        model = RandomForestClassifier(n_estimators=1000,min_samples_leaf=2,
                                        min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                                      random_state=25)
        model.fit(tfidf_train,Y_train) 
        predictions_rf = model.predict(tfidf_test)
        # Use accuracy_score function to get the accuracy
        print("rf Accuracy Score -> ",accuracy_score(predictions_rf, Y_test)*100)
        print(classification_report(Y_test,predictions_rf))

        return model
    else:
        print("No invalid model selected, try again")
        return 0


In [9]:
X_train, Y_train, X_test, Y_test = read_splitted_data()
Tfidf_vect, tfidf_train, tfidf_test = tf_idf_config(X_train, X_test)
selected = model_selection(model_name = "LGBM", tfidf_train = tfidf_train, tfidf_test = tfidf_test, Y_train = Y_train, Y_test = Y_test)

C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ligbm Accuracy Score ->  87.71929824561403
              precision    recall  f1-score   support

          -1       0.85      0.85      0.85        46
           1       0.90      0.90      0.90        68

    accuracy                           0.88       114
   macro avg       0.87      0.87      0.87       114
weighted avg       0.88      0.88      0.88       114



### Test Prediction

In [10]:
def lower_df(df):
    df['text'] = df['text'].str.lower()
    return df

In [11]:
def segmentation(df):


    '''UNDER THE SEA'''
    
    list_text = df['text'].to_list()
    #print(list_text[0])
    for i in range(len(list_text)):
        list_text[i] = word_tokenize(list_text[i], format='text')
    
    df['text'] =  list_text
    return df

In [12]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [13]:
def remove_stopwords(df):
    stopwords = get_stopwords_list(stopwords_path)

    list_text = df['text'].tolist()

    results = []
    for text in list_text:
        tmp = text.split(' ')
        for stop_word in stopwords:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))

    df['text'] = results
    return df

In [14]:
def data_preprocessing(df):
    df = lower_df(df)
    df = segmentation(df)
    df = remove_stopwords(df)

    return df

In [15]:
def predict_sample(tmp_text):
    input = tmp_text
    list_input = [input]
    test_df = pd.DataFrame()
    test_df['text'] = list_input
    test_df = data_preprocessing(test_df)

    test_df = Tfidf_vect.transform(test_df['text'])
    test_df = test_df.toarray()
    prediction = selected.predict(test_df)   

    if (prediction[0] == -1):
        print("Bai viet co kha nang tieu cuc cao !!!")   
    else:
        print("Bai viet co kha nang tieu cuc thap :3")

In [22]:
input  = """Người dân ở chung cư vì giá tiền hợp lý và có nhiều tiện ích. Khó xác định thuế đất Hiện TP Hồ Chí Minh có không ít chung cư vì vướng việc xác định thuế đất nên chưa được cấp sổ hồng. Mới đây, các cư dân tại chung cư Gateway Thảo Điền (phường Thảo Điền, thành phố Thủ Đức, TP Hồ Chí Minh) đã có đơn phản ánh về việc chậm cấp sổ hồng cho cư dân. Theo đó, đa số người dân nhận nhà từ tháng 4/2018 và đã thanh toán 95% giá trị hợp đồng, nhưng đến nay vẫn chưa được cấp sổ hồng. Nguyên nhân khiến chung cư này chưa được cấp sổ hồng là do dự án có diện tích khối đế vượt diện tích tầng hầm nên chưa xác định được thuế đất bị vượt. Chủ đầu tư dự án này cũng đã nhiều lần có văn bản xin được đóng tiền sử dụng đất bổ sung để cơ quan chức năng cấp sổ hồng cho cư dân nhưng không được các cơ quan chức năng chấp nhận vì chưa giải quyết được các thủ tục hành chính. Tương tự, chung cư Minh Thành (Quận 7) được đưa vào sử dụng từ năm 2008 và đến nay tất cả chủ sở hữu căn hộ cũng vẫn chưa được nhận sổ hồng, dù đã đóng hết tiền mua nhà. Ông Bùi Ngọc Minh, Giám đốc Công ty Minh Thành cho biết, hồ sơ cấp sổ hồng cho cư dân bị vướng do còn đang bị nhập nhằng việc xác định đất ở hay thương mại. Sau nhiều lần doanh nghiệp phản ánh, lãnh đạo TP Hồ Chí Minh và Quận 7 đã tổ chức nhiều cuộc họp giải quyết nhưng chưa có kết quả. Vì vậy, ông Bùi Ngọc Minh đề nghị Sở Tài nguyên và Môi trường TP Hồ Chí Minh sớm hoàn thành thủ tục liên quan để công ty nộp hồ sơ xin cấp sổ hồng cho cư dân. Việc chậm trễ khiến giá trị thương mại căn hộ giảm, người dân gặp khó khăn khi muốn mua bán, sang nhượng, vay vốn ngân hàng... Dân số cơ học tăng nhanh khiến nhu cầu về nhà ở của người dân cũng tăng cao tại TP Hồ Chí Minh. Thống kê tại Tập đoàn Hưng Thịnh, đến nay có rất nhiều dự án dù đã giao nhà và cư dân vào ở đã lâu nhưng sổ hồng vẫn chưa được cấp, nguyên nhân do còn vướng mắc từ các quy định của nhà nước. Ông Trần Quốc Dũng, Phó Tổng giám đốc Tập đoàn Hưng Thịnh cho biết, doanh nghiệp đang có 13 dự án với 8.791 căn hộ chưa được cấp sổ hồng, hầu hết vướng mắc ở khâu xác định tiền sử dụng đất. Những tắc nghẽn này hiện nằm ở Sở Tài nguyên và Môi trường TP Hồ Chí Minh. Ví dụ như chung cư Lavita Garden (thành phố Thủ Đức), khi thủ tục thẩm định giá đất được triển khai từ cuối năm 2015, Sở Tài Nguyên và môi trường đã 4 lần trình phương án giá đất lên Hội đồng thẩm định giá đất TP Hồ Chí Minh, song đến nay vẫn chưa được thông qua. Để giải quyết những khó khăn, vướng mắc liên quan đến việc tính thuế đất, theo ông Nguyễn Toàn Thắng, Giám đốc Sở Tài Nguyên và Môi trường TP Hồ Chí Minh, khâu xác định giá đất của dự án để người sử dụng đất thực hiện nghĩa vụ tài chính về đất đai với nhà nước là khâu khá quan trọng. Thực tế có hàng chục dự án bị “treo” sổ hồng chỉ vì bị "tắc" tiền sử dụng đất của dự án, doanh nghiệp. Trong đó, có 7 vấn đề liên quan đến việc khó thẩm định giá đất như: Vướng mắc liên quan đến việc tổ chức thuê đơn vị tư vấn thẩm định giá và thời hạn hiệu lực của Chứng thư thẩm định giá; việc xác định giá đất trong quá khứ; vướng mắc trong các phương pháp xác định giá đất khi thực hiện thẩm định giá; việc định giá đất đối với một số vị trí có lợi thế sinh lợi cao, ở vị trí đắc địa và khó khăn trong việc cấn trừ các khoản chi phí tạo lập đất vào nghĩa vụ tài chính. Cần phải giải quyết sớm Theo ông Nguyễn Toàn Thắng, từ khi Luật Đất đai 2013 có hiệu lực, trên địa bàn thành phố có 232 dự án nhà ở và cơ quan chức năng đã cấp sổ hồng trên 71.000 căn hộ. Điều này chứng minh tất cả các dự án nhà ở đủ điều kiện đều được giải quyết. Trong số 50.000 căn hộ chưa cấp được sổ hồng là do có nhiều dự án xây dựng sai phép. Hiện Sở đang phối hợp với Sở Xây dựng xử lý 6 dự án nhà ở xây dựng sai phép để nghiệm thu và cấp sổ hồng. Ngoài ra, nhiều dự án chủ đầu tư chưa thực hiện đầy đủ trách nhiệm. Theo đó, các dự án hoàn thành phải nộp hồ sơ pháp lý và các vấn đề liên quan nghĩa vụ tài chính nhưng chủ đầu tư không nộp, nên không thể giải quyết. Vừa qua, Sở cũng đã xử phạt nhiều trường hợp chậm nộp hồ sơ hành chính để được cấp sổ hồng cho người dân. "TP Hồ Chí Minh sẽ tiếp tục ghi nhận những khó khăn mà doanh nghiệp và sẽ nỗ lực để giải quyết cho doanh nghiệp. Thời gian qua, các doanh nghiệp bất động sản đã đồng hành cùng TP Hồ Chí Minh, góp phần làm thay đổi diện mạo hạ tầng cũng như góp phần không nhỏ vào nguồn thu ngân sách của TP Hồ Chí Minh. Do đó, với dự án hoàn chỉnh pháp lý, Sở sẽ giải quyết ngay việc cấp sổ hồng. Đối với những vướng mắc phát sinh thì ở cấp độ TP Hồ Chí Minh, Sở đã kiến nghị UBND TP Hồ Chí Minh để xem xét, chấp thuận tiếp tục tổ chức buổi họp với các sở, ngành, đơn vị có liên quan về những khó khăn, vướng mắc trong công tác xác định giá đất cụ thể và giải pháp tháo gỡ kịp thời", ông Nguyễn Toàn Thắng cho biết thêm. Cư dân ở chung cư Lavita Garden (thành phố Thủ Đức) chờ đợi nhiều năm vẫn chưa có sổ hồng. Trong khi đó, ông Lê Hoàng Châu, Chủ tịch Hiệp hội Bất động sản TP Hồ Chí minh cho biết, TP Hồ Chí Minh đã có nhiều cuộc họp trực tiếp với doanh nghiệp, người dân để giải quyết vấn đề chậm cấp sổ hồng cho các chung cư nhưng việc này vẫn bế tắc. Trong đó, có điểm chung là hồ sơ trình lên Sở Tài Nguyên và Môi trường TP Hồ Chí Minh xin được cấp sổ hồng rất chậm và đa số bị vướng nên dù doanh nghiệp muốn được cấp sổ hồng nhưng vẫn chưa thể đóng được tiền sử dụng đất. Thậm chí, nhiều doanh nghiệp quá sốt sắng, chủ động nộp trước tiền sử dụng đất, có doanh nghiệp đã đóng đến mức cao nhất theo đề xuất của Sở Tài nguyên và Môi trường nhưng vẫn không giải quyết được vấn đề. Ví dụ như Công ty Cơ khí xây dựng Bình Triệu sau nhiều lần trình duyệt giá đất đã chủ động ứng trước 50% tiền sử dụng đất (khoảng 168 tỉ đồng); Tập đoàn Novaland cũng ứng 500 tỉ đồng nộp trước tiền sử dụng đất… với mong muốn nhanh chóng có sổ cho người mua nhà nhưng vẫn không được các cơ quan chức năng giải quyết. Theo ông Lê Hoàng Châu, để giải quyết tình trạng chậm cấp sổ hồng liên quan đến việc xác định thuế đất, vừa qua, Hội đồng thẩm định giá đất đã có đề xuất với UBND TP Hồ Chí Minh phương thức tính tiền sử dụng đất nên áp dụng hệ số điều chỉnh giá đất (hệ số K). Nếu áp dụng được hệ số K này thì việc tính tiền sử dụng đất đang kéo dài 3 - 5 năm chỉ còn 10 - 15 ngày làm việc. Điều này liên quan đến việc phải sửa đổi khoản 2 điều 18 Nghị định 44 năm 2014 của Chính phủ. Vì vậy, Hiệp hội cũng đã có kiến nghị Chính phủ giao toàn quyền cho cấp tỉnh ban hành bảng giá đất phù hợp với điều kiện của địa phương và hệ số K. Nếu những điều này được thông qua, việc tính tiền sử dụng đất sẽ được triển khai nhanh chóng, dễ dàng hơn. Từ đó công tác cấp sổ hồng tại các chung cư cũng đẩy nhanh hơn rất nhiều so với hiện nay.
"""
predict_sample(input)


Bai viet co kha nang tieu cuc cao !!!
